In [186]:
import csv
import datetime as datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [187]:
def get_url(location):
    """Generate URL from position and Location
    :parameter: location -> Country"""
    template = 'https://pt.indeed.com/ofertas?jt=all&st=&radius=25&l={}&fromage=any&limit=50&sort=date&psf=advsrch&from=advancedsearch&start=0'
    url = template.format(location)
    return url

In [188]:
url = get_url(location='Portugal')
url

'https://pt.indeed.com/ofertas?jt=all&st=&radius=25&l=Portugal&fromage=any&limit=50&sort=date&psf=advsrch&from=advancedsearch&start=0'

# Extract Raw Html

In [189]:
response = requests.get(url)
response

<Response [200]>

In [190]:
soup = BeautifulSoup(response.text, 'html.parser')

In [191]:
refs = soup.find_all('a', {'rel':'nofollow', 'data-hiring-event':'false'})
ref = refs[0]

In [192]:
cards = soup.find_all('div', 'job_seen_beacon')

In [193]:
len(cards)

50

# Prototype the model with a single record

In [53]:
card = cards[0]

In [54]:
jobId = ref.get('data-jk')
jobUrl = 'https://pt.indeed.com/ver-oferta?jk={}&tk=1ffnjle1itv8s800&from=serp&vjs=3'.format(jobId)

In [55]:
jobTitle = card.h2.find_all("span")[1].text
jobTitle

'Atendimento ao Cidadão – Refª. 2734'

In [56]:
companyName = card.find_all('span', class_='companyName')[0].string
companyName


'Techinfor'

In [66]:
jobLocation = card.find_all('div', class_='companyLocation')[0].contents[0]
jobLocation

'Lisboa'

In [81]:
shortSummary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
shortSummary 

'Contrato de Trabalho com Vencimento base + Subsídio de Alimentação ´+ Prémio. Possibilidade de integração num projeto de grande visibilidade Nacional.'

In [58]:
try:
    jobSalary = card.find('span', class_='salary-snippet').string[:-8]
except AttributeError:
    jobSalary = ''
jobSalary


''

In [59]:
postDate = card.find('span', 'date').string
today = datetime.date.today().strftime('%Y-%m-%d')

## Get full Job Description

In [27]:
response = requests.get(jobUrl)
response

<Response [200]>

In [28]:
soup = BeautifulSoup(response.text, 'html.parser')

In [29]:
jobDescription = soup.find('div', 'jobsearch-jobDescriptionText').text.strip().replace('\n', ' ')

In [30]:
# originalJobLink
try:
    originalJobLink = soup.find('div', {'id':"originalJobLinkContainer"}).a['href']
except AttributeError:
    originalJobLink = ''

In [ ]:
def get_jobDescription(jobUrl):
    response = requests.get(jobUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    jobDescription = soup.find('div', 'jobsearch-jobDescriptionText').text.strip().replace('\n', ' ')
    
    # originalJobLink
    try:
        originalJobLink = soup.find('div', {'id':"originalJobLinkContainer"}).a['href']
    except AttributeError:
        originalJobLink = ''
    return jobDescription, originalJobLink

# Generalize the Model with a function

In [ ]:
def get_record(card):
    """Extract Job dat from a single record"""
    
    # job Id and job Url
    jobId = ref.get('data-jk')
    jobUrl = 'https://pt.indeed.com/ver-oferta?jk={}&tk=1ffnjle1itv8s800&from=serp&vjs=3'.format(jobId)

    # job title
    jobTitle = card.h2.find_all("span")[1].text

    # company name
    companyName = card.find_all('span', class_='companyName')[0].string

    # job location
    jobLocation = card.find_all('div', class_='companyLocation')[0].contents[0]

    # Short Summary
    shortSummary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')

    # job salary
    try:
        jobSalary = card.find('span', class_='salary-snippet').string[:-8]
    except AttributeError:
        jobSalary = ''
    
    # posting date and scraping date
    postDate = card.find('span', 'date').string
    today = datetime.date.today().strftime('%Y-%m-%d')
    
    jobDescription, originalJobLink = get_jobDescription(jobUrl)

    record = (jobId, jobTitle, companyName, jobLocation, shortSummary, jobSalary, today, postDate, jobUrl, jobDescription, originalJobLink)
    return record 

# Iterating By Page

In [64]:
while True:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    refs = soup.find_all('a', {'rel':'nofollow', 'data-hiring-event':'false'})
    cards = soup.find_all('div', 'job_seen_beacon')

    for card in cards:
        record = get_record()
        records.append(record)

    try:
        url = 'https://pt.indeed.com' + soup.find('a', {'aria-label':'Próxima'}).get('href')
    except AttributeError:
        break

# Putting All together

In [1]:
import csv
import datetime as datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import random
import os


def get_url(location, start):
    """Generate URL from position and Location
    :parameter: location -> Country"""
    template = 'https://pt.indeed.com/ofertas?jt=all&st=&radius=25&l={}&fromage=any&limit=50&sort=date&psf=advsrch&from=advancedsearch&start={}'
    url = template.format(location, start)
    return url

def get_jobDescription(jobUrl):
    response = requests.get(jobUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        jobDescription = soup.find('div', 'jobsearch-jobDescriptionText').text.strip().replace('\n', ' ')
    except AttributeError:
        jobDescription = ''

    # originalJobLink
    try:
        originalJobLink = soup.find('div', {'id':"originalJobLinkContainer"}).a['href']
    except AttributeError:
        originalJobLink = ''

    return jobDescription, originalJobLink

def get_record(card, ref):
    """Extract Job dat from a single record"""
    
    # job Id and job Url
    jobId = ref.get('data-jk')
    jobUrl = 'https://pt.indeed.com/ver-oferta?jk={}&tk=1ffnjle1itv8s800&from=serp&vjs=3'.format(jobId)

    # job title
    try:
        jobTitle = card.h2.find_all("span")[1].text
    except AttributeError:
        jobTitle = ''

    # company name
    try:
        companyName = card.find('span', class_='companyName').string
    except AttributeError:
        companyName = ''

    # job location
    try:
        jobLocation = card.find('div', class_='companyLocation').contents[0]
    except:
        jobLocation = ''

    # Short Summary
    try:
        shortSummary = card.find('div', 'job-snippet').text.strip().replace('\n', ' ')
    except AttributeError:
        shortSummary = ''
        
    # job salary
    try:
        jobSalary = card.find('span', class_='salary-snippet').string[:-8]
    except AttributeError:
        jobSalary = ''
    
    # posting date and scraping date
    postDate = card.find('span', 'date').string
    today = datetime.date.today().strftime('%Y-%m-%d')
    
    jobDescription, originalJobLink = get_jobDescription(jobUrl)

    record = (jobId, jobTitle, companyName, jobLocation, shortSummary, jobSalary, today, postDate, jobUrl, jobDescription, originalJobLink)
    return record 

def main(location, start):
    """Run the main program routine"""
    records = []
    url = get_url(location, start)

    # Extract Job Data
    while True:
        # Random Sleep Time 0 to 4 seconds
        time.sleep(random.random()*20)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        refs = soup.find_all('a', {'rel':'nofollow', 'data-hiring-event':'false'})
        cards = soup.find_all('div', 'job_seen_beacon')

        for idx, card in enumerate(cards):
            record = get_record(card, refs[idx])
            records.append(record)

        try:
            url = 'https://pt.indeed.com' + soup.find('a', {'aria-label':'Próxima'}).get('href')
        except AttributeError:
            print(url)
            break

    # Save Job Data
    if os.path.exists('indeed_jobs.csv') == False:
        with open('indeed_jobs.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Id', 'JobTitle', 'Company', 'Location', 'ShortSummary', 'Salary', 'ExtractDate', 'PostDate', 'JobUrl', 'FullDescription', 'CompanyLink'])
            writer.writerows(records)
        f.close()
    else:
        with open('indeed_jobs.csv', 'a+', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(records)
        f.close()
    
    print(f'Scrapped {len(records)} job posting')

In [2]:
start = len(pd.read_csv('indeed_jobs.csv'))
main('Portugal', start)

IndexError: list index out of range

In [ ]:
print(f'Number of scraped Job Ads -> {len(pd.read_csv("indeed_jobs.csv"))}')
pd.read_csv('indeed_jobs.csv')

Number of scraped Job Ads -> 1000


,Id,JobTitle,Company,Location,ShortSummary,Salary,ExtractDate,PostDate,JobUrl,FullDescription,CompanyLink
0,ca973c0cd8100998,Operador Gasolineira,Auchan,Lisboa,Na Auchan Retail Portugal o cliente é o centro...,NaN,9/17/2021,Agora mesmo,https://pt.indeed.com/ver-oferta?jk=ca973c0cd8...,Vem descobrir como é trabalhar no comércio. Na...,https://pt.indeed.com/rc/clk?jk=ca973c0cd81009...
1,4.34691E+15,Transactional Support Junior Specialist,Nestlé,Linda-a-Velha,This professional will be responsible to execu...,NaN,9/17/2021,Agora mesmo,https://pt.indeed.com/ver-oferta?jk=4346909910...,"Position Snapshot Location: Linda-a-Velha, Li...",https://pt.indeed.com/rc/clk?jk=43469099100027...
2,8800866b699bfe57,Consultoria,Via Consulting,Portugal,Procuramos pessoas com formação superior (lice...,NaN,9/17/2021,Agora mesmo,https://pt.indeed.com/ver-oferta?jk=8800866b69...,Procuramos pessoas com formação superior (lice...,NaN
3,c8eb1a51935523cf,R&D,Via Consulting,Portugal,Procuramos pessoas com formação (licenciatura ...,NaN,9/17/2021,Agora mesmo,https://pt.indeed.com/ver-oferta?jk=c8eb1a5193...,Procuramos pessoas com formação (licenciatura ...,NaN
4,da9e05f1c0278071,Vendedor (m/f),Sonae,Portugal,. Acompanhamento e aconselhamento dos nossos c...,NaN,9/17/2021,Agora mesmo,https://pt.indeed.com/ver-oferta?jk=da9e05f1c0...,Na Direção de Vendas temos como missão garanti...,https://pt.indeed.com/rc/clk?jk=da9e05f1c02780...
...,...,...,...,...,...,...,...,...,...,...,...
995,4f786cd8ad7d8ddf,"Support Specialist, Trilogy (Remote) - $30,000...",Crossover,Lisboa,We make weekly investments in our team’s profe...,NaN,2021-09-18,há 4 dias,https://pt.indeed.com/ver-oferta?jk=4f786cd8ad...,NaN,NaN
996,96ebfc6d338a3711,"Subsystem CTO, Trilogy (Remote) - $200,000/yea...",Crossover,<span>Remoto</span>,Each week you will have a different design pro...,NaN,2021-09-18,há 4 dias,https://pt.indeed.com/ver-oferta?jk=96ebfc6d33...,NaN,NaN
997,caffaf953afd5362,"Inside Sales Representative, Trilogy (Remote) ...",Crossover,Lisboa,It is time to move on from deceptive and archa...,NaN,2021-09-18,há 4 dias,https://pt.indeed.com/ver-oferta?jk=caffaf953a...,NaN,NaN
998,787c03babc40e3bf,Team Leader (German speaker),TIME FRAME,Porto,"As specialists in customer care, we use the la...",NaN,2021-09-18,há 4 dias,https://pt.indeed.com/ver-oferta?jk=787c03babc...,NaN,NaN
